In questa parte ci occupiamo della parte 'demografica' della Serie A. Adremo ad analizzare come son distribuite le età, le nazionalità ed i minuti giocati, per gli attaccanti. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import scipy.stats as stats
import tensorflow as tf
from scipy.interpolate import make_interp_spline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

# Elenco dei file da leggere
files = ['attaccanti_23.xlsx', 'attaccanti_22.xlsx', 'attaccanti_21.xlsx', 'attaccanti_20.xlsx', 'attaccanti_19.xlsx']

# Faccio un dizionario per mappare i file agli anni
year_mapping= {'attaccanti_23.xlsx': 2023,'attaccanti_22.xlsx': 2022,'attaccanti_21.xlsx': 2021,'attaccanti_20.xlsx': 2020,
                'attaccanti_19.xlsx': 2019}

Dopo aver importato i file necessari andiamo ad analizzare come sono variate le età nei 5 anni analizzati, visualizziamo il tutto con un diagramma a torta.

In [ ]:
# Definisci le fasce di età e le etichette
bins = [15, 20, 25, 30, 35, 40]
labels = ['15-20', '21-25', '26-30', '31-35', '36-40']

# Dizionario per conservare i conteggi annuali per ciascuna fascia d'età
fasce_eta_conti = {label: [] for label in labels}
medie_totali_annuali = []
deviazioni_annuali = []  # Nuova lista per memorizzare le deviazioni standard

# Ciclo sui file per calcolare la numerosità per ciascuna fascia, la media totale e la deviazione standard
for file in files:
    data = pd.read_excel(file)
    eta = data['Age']
    
    # Categorizza le età nelle fasce specificate
    eta_bins = pd.cut(eta, bins=bins, labels=labels, right=True, include_lowest=True)
    
    # Conta il numero di giocatori per ciascuna fascia di età e salva nel dizionario
    for label in labels:
        conteggio_eta_fascia = (eta_bins == label).sum()
        fasce_eta_conti[label].append(conteggio_eta_fascia)
    
    # Calcola la media e la deviazione standard dell'età per l'anno
    media_eta_totale = eta.mean()
    deviazione_eta_totale = eta.std(ddof=1)  # ddof=1 per deviazione standard campionaria
    
    # Aggiungi i valori alla lista
    medie_totali_annuali.append(media_eta_totale)
    deviazioni_annuali.append(deviazione_eta_totale)

# Numero di anni considerati
anni = ['2019', '2020', '2021', '2022', '2023']

# Creazione del grafico a barre (numerosità per fascia di età e anno)
fig, ax = plt.subplots(figsize=(10, 8))
bar_width = 0.8
mini_bar_width = bar_width / len(anni)
x_pos = np.arange(len(labels))

for i, anno in enumerate(anni):
    conti_annuali = [fasce_eta_conti[label][i] for label in labels]
    offset = (i - (len(anni) - 1) / 2) * mini_bar_width
    ax.bar(x_pos + offset, conti_annuali, mini_bar_width, label=anno)

ax.set_xlabel('Fascia di Età')
ax.set_ylabel('Numero di Giocatori')
ax.set_title('Numero di Attaccanti per Fascia di Età e Anno')
ax.set_xticks(x_pos)
ax.set_xticklabels(labels)
ax.legend(title='Anno')

max_y = max(max(conti_annuali) for conti_annuali in fasce_eta_conti.values())
ax.set_yticks(np.arange(0, max_y + 5, 5))

plt.show()

# Creazione 2 grafico
fig, ax = plt.subplots(figsize=(10, 6))

# Traccia l'andamento delle medie totali annuali
ax.plot(anni, medie_totali_annuali, marker='o', color='k', label='Media Totale')

# Aggiungi le deviazioni standard come barre di errore
ax.errorbar(anni, medie_totali_annuali, yerr=deviazioni_annuali, fmt='o', color='k', label='Deviazione Standard')

ax.set_xlabel('Anno')
ax.set_ylabel('Media Età')
ax.set_title("Andamento della Media Totale dell'Età dal 2019 al 2023")

ax.set_ylim(15,40)
ax.set_yticks(np.arange(15,40,1))

ax.grid(True)
ax.legend()

plt.show()

Passiamo ora all'analisi delle nazionalità. Vediamo quali sono le nazionalità più presenti in Serie A.

In [ ]:
# Ciclo sui file per generare i grafici
for file in files:

    data = pd.read_excel(file)
     
    nazionalita = data['Nation']   #istogramma nazionalità

    # Conta il numero di calciatori per ciascuna nazionalità
    nazionalita_counts = nazionalita.value_counts()

    # Crea l'istogramma
    fig, ax = plt.subplots(figsize=(10, 6))
    nazionalita_counts.plot(kind='bar', ax=ax, color='skyblue')

    # Aggiungi i titoli e le etichette
    ax.set_title(f'Distribuzione delle Nazionalità degli attaccanti Anno: {year_mapping[file]}')
    ax.set_xlabel('Nazionalità')
    ax.set_ylabel('Numero di calciatori')

    # Ruota le etichette sull'asse x per una migliore leggibilità
    plt.xticks(rotation=45, ha='right')

    # Mostra il grafico
    plt.tight_layout()
    plt.show()

In [ ]:
for file in files:
 data = pd.read_excel(file)

 # Estrai i dati rilevanti (calciatori e valori nella settima colonna)
 calciatori = data['Player']  # Assume che i nomi dei calciatori siano nella prima colonna
 valori = data['MP']      # Settima colonna partendo da sinistra

 filtrato = valori > 20
 calciatori_filtrati = calciatori[filtrato]
 valori_filtrati = valori[filtrato]

 # Crea un istogramma
 plt.figure(figsize=(12, 8))
 plt.bar(calciatori_filtrati, valori_filtrati, color='skyblue', edgecolor='black')

 # Ottimizza il layout
 plt.xlabel('Calciatori', fontsize=14)
 plt.ylabel('Valori', fontsize=14)
 plt.title(f"Minuti giocati dai Calciatori divisi per 90' Anno: {year_mapping[file]}")
 plt.xticks(rotation=45, ha='right', fontsize=10)
 plt.yticks(fontsize=12)

 # Aggiungi una curva di fit
 x = np.arange(len(calciatori_filtrati))  # Converti l'indice dei calciatori filtrati in numerico
 spl = make_interp_spline(x, valori_filtrati, k=3)  # Fit con spline
 x_smooth = np.linspace(x.min(), x.max(), 500)  # Genera un range di valori lisciato
 y_smooth = spl(x_smooth)

 plt.plot(x_smooth, y_smooth, color='red', linewidth=2, label='Fit Curva')

 plt.tight_layout()
 plt.legend()
 plt.show()

 media_valori_filtrati = valori_filtrati.mean()

 # Mostra il risultato
 print(f"La media per l'Anno {year_mapping[file]} è:{media_valori_filtrati}")

Aggiungiamo una cella di codice che stampa gli istogrammi di tutte le statistiche di tutti 5 gli anni. In modo da poterli 
analizzare singolarmente in seguito, senza appesantire il codice con tutti i grafici.

In [ ]:
for file in files:
 data = pd.read_excel(file)

# Selezionare solo le colonne numeriche
numeric_data = data.select_dtypes(include=[np.number])

# Funzione per calcolare il numero di bin con la regola di Freedman-Diaconis
def freedman_diaconis_bins(data):
    q25, q75 = np.percentile(data, [25, 75])
    iqr = q75 - q25
    if iqr == 0:  # Evitare divisioni per zero
        return 20  # Numero di bin di default in caso di IQR nullo
    bin_width = 2 * iqr / (len(data) ** (1 / 3))
    if bin_width == 0:
        return 5  # Numero di bin di default in caso di width troppo piccolo
    return int(np.ceil((data.max() - data.min()) / bin_width))

# Creare istogrammi per ogni colonna numerica con controllo su variazioni minime
for column in numeric_data.columns:
    col_data = numeric_data[column].dropna()  # Rimuovi eventuali valori NaN
    if col_data.empty or col_data.max() == col_data.min():
        continue
    
    # Calcolare il numero di bin con la regola di Freedman-Diaconis
    bins = freedman_diaconis_bins(col_data)
    
    # Creare l'istogramma
    plt.figure(figsize=(10, 6))
    plt.hist(col_data, bins=bins, density=False, color='k', edgecolor = 'r', label='Dati')

    # Titolo e etichette
    plt.title(f'Istogramma della colonna: {column}')
    plt.xlabel(f'{column}')
    plt.ylabel('Numero di giocatori')

    # Printa tutti gli istogrammi, commentare le righe sotto per evitare
    plt.show()
    media = col_data.mean()
    print(f"La media di {column} per l'anno {year_mapping[file]} è: {media:.2f}")

Creiamo una funzione che, dati in input la statistica e l'anno desiderato, restituisce l'istogramma di quella statistica
relativa a quell'anno

In [ ]:
def istogramma(statistica, anno):
    # Verifica che l'anno esista nel mapping
    if anno not in year_mapping:
        print(f"L'anno {anno} non è valido. Gli anni disponibili sono: {list(year_mapping.keys())}")
        return

    # Carica il dataset corrispondente all'anno
    file = year_mapping[anno]
    data = pd.read_excel(file)

    # Seleziona la colonna specificata
    if statistica not in data.columns:
        print(f"La statistica '{statistica}' non è presente nei dati.")
        return
    
    col_data = data[statistica].dropna()  # Rimuovi eventuali valori NaN

    if col_data.empty or col_data.max() == col_data.min():
        print(f"La colonna '{statistica}' non ha abbastanza variabilità per creare un istogramma.")
        return

    # Calcolare il numero di bin con la regola di Freedman-Diaconis
    def freedman_diaconis_bins(data):
        q25, q75 = np.percentile(data, [25, 75])
        iqr = q75 - q25
        if iqr == 0:  # Evitare divisioni per zero
            return 20  # Numero di bin di default in caso di IQR nullo
        bin_width = 2 * iqr / (len(data) ** (1 / 3))
        if bin_width == 0:
            return 5  # Numero di bin di default in caso di width troppo piccolo
        return int(np.ceil((data.max() - data.min()) / bin_width))

    # Calcolare il numero di bin
    bins = freedman_diaconis_bins(col_data)

    # Creare l'istogramma
    plt.figure(figsize=(10, 6))
    plt.hist(col_data, bins=bins, density=False, color='k', edgecolor='r', label='Dati')

    # Titolo e etichette con l'anno specificato
    plt.title(f'Istogramma della colonna: {statistica} ({anno})')
    plt.xlabel(f'{statistica}')
    plt.ylabel('Numero di giocatori')

    # Mostrare il grafico
    plt.show()

    # Calcola e stampa la media della colonna
    media = col_data.mean()
    print(f"La media di {statistica} per l'anno {anno} è: {media:.2f}")
    giocatore_max = data.loc[data[statistica].idxmax()]['Player']
    max_value = col_data.max()
    print(f"Il giocatore che ha registrato il maggior numero di {statistica} è {giocatore_max} con {max_value:.2f}.")

Con questa funzione possiamo richiamare dei grafici specifici e analizzarli se ci interessa.

In [ ]:
statistica = 'Won'
anno = 2019
istogramma(statistica,anno)

CORRELAZIONI

INTRODUZIONE

Il calcio moderno è diventato molto caro, sopratutto il reparto di attacco. Le società chiedono tantissimi soldi per un giocatore "solamente" perchè ha realizzato molti Gol o Assist nella stagione precedente. Non tutte le squadre, però, hanno una disponibilità economica elevata per comprare un attaccante già affermato. La nostra tesi è che esistano delle correlazioni, fra le statistiche sopracitate e le altre di gioco, che una squadra può andare a studiare per stimare il numero di Gol e Assist che può realizzare un attaccante. In questo modo una società non sarà costretta ad affidarsi al numero di Gol ed Assist e potrà comprare giocatori altrettanto validi ad un prezzo minore, risparmiando soldi.

DATA CLEANING

Prima di procedere con l'analisi bisogna fare un processo di 'data cleaning' ovvero una pulizia del dataframe, aggiungendo colonne che possono essere utili oppure togliendone altre che non vogliamo prendere in considerazione.

Eliminiamo cartellini gialli e rossi collezionati dai giocatori, in quanto rappresentano la disciplina di questi e non le loro capacità balistiche. Dopodiche eliminiamo le colonne relative ai calci piazzati in quanto rigori e punizioni hanno i propri specialisti scelti dall'allenatore, non sarebbe quindi equo considerare una statistica alla quale la maggior parte dei giocatori non prendono neanche parte.

Ovviamente scartiamo le celle relative al Nome del giocatore, la Squadra per cui gioca e la sua Nazionalità. Subito dopo togliamo quelle colonne che rappresentano "operazioni" fra altre statistiche, come ad esempio somme fra due dati, percentuali o rapporti, in quanto non sono statistiche di gioco (realizzate sul campo dal giocatore) ma create ad hoc per altri tipi di analisi non inerenti al nostro.

Queste statistiche non verranno mai considerate in questa parte del progetto.

Suddivideremo le statistiche del database in 4 categorie e le analizzeremo una ad una.

1) STATISTICHE DI IMPIEGO

La prima categoria riguarda le statistiche non di campo, quelle relative ad età, minutaggio e partite giocate

In [ ]:
#Crea 5 matrici colorate in cui ogni quadratino rappresenta l'indice di 
# correlazione fra le statistiche per annata

correlations = []
years = []

colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad','G+A', 'G-PK', 'PK',
       'PKatt', 'CrdY', 'CrdR', 'xG', 'PrgP', 'PrgR', 'Sh', 'SoT', 'SoT%',
       'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG', 'Cmp', 'Att', 'Cmp%', 'xA', 'KP',
       'PPA', 'CrsPA', 'SCA', 'GCA', 'Tkl', 'TklW', 'Mid 3rd', 'Att 3rd',
       'Touches', 'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ', 'Succ%',
       'Carries', 'TotDist', '2CrdY', 'Fld', 'Off', 'Crs', 'Duels', 'Won', 'Lost',
       'Won%']

for file in files:
    
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost'] # Creo la colonna dei duelli aerei, può essere utile per l'analisi

    # Uso .drop per rimuovere dal data frame quelle statistiche che non voglio usare in questa analisi
    data_frame.drop(columns=colonne_da_eliminare, inplace=True) 

    corr_matrix = data_frame.corr()  # Calcolo la matrice di correlazione

    year = year_mapping[file]  # Ottengo l'anno dal file usando il mapping
    years.append(year)

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}", zmin = -1, zmax = 1)
    # Ottengo in output il grafico di correlazione

    fig.show()

# Inserisco questa riga di codice in modo da poter visualizzare le colonne usate per l'analisi
print(f'Le colonne usate per questa analisi sono {list(data_frame.columns)}')

Il codice stampa delle matrici interattive quadrate simmetriche dove ogni cella è colorata in base all'indice di correlazione delle statistiche scritte sulla riga e colonna corrispondente. Ovviamente la diagonale ha tutti i valori uguali ad uno in quanto indica la correlazione di una statistica con se stessa.
Diciamo che è interattiva in quanto mettendo il cursore su un quadrato vengono indicate le statistiche e l'indice fra di loro.
Sulla destra è riportata una barra che fa da legenda, con la quale si può associare al colore il valore dell'indice, vengono impostati estremi fissi e non variabili durante tutte l'analisi in modo che i colori possano aiutarci in una fase preliminare dell'analisi.

L'indice di correlazione è un numero compreso tra -1 e 1 che indica la relazione fra due variabili. Un indice di correlazione positivo indica che all'aumentare di una statistica anche l'altra lo fa. Mentre quello negativo indica che all'aumentare di una l'altra diminuisce. Più questo valore è vicino all'uno, in modulo, più la relazione è forte, più ci si avvicina allo zero più è debole

A noi importa analizzare gli indici delle statistiche con Gol ed Assist, quindi guardiamo le ultime due righe (colonne). 
Con la matrice possiamo farci un'idea preliminare, grazie ai colori, dell'indice di correlazione fra le statistiche.

In [ ]:
# Creo una funzione che mi permette di scoprire quali sono le statistiche meglio e peggio correlate con una desiderata

def best_worst(stat):
 for file in files:
    year = year_mapping[file]
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost']

    data_frame.drop(columns=colonne_da_eliminare, inplace=True) 

    corr_matrix = data_frame.corr()

    if stat in corr_matrix.columns:
        # Ordina le correlazioni con la statistica scelta
        sorted_corr = corr_matrix[stat].sort_values(ascending=False)

        # Trova le 5 migliori e peggiori correlazioni (cambiare i range per saperne di più o di meno)
        best_corr = sorted_corr[1:6]  # Parto da 1 per evitare la correlazione della statistica con se stessa
        worst_corr = sorted_corr[-5:]

        print(f"Anno {year}: Le 5 statistiche meglio correlate con {stat} sono:\n{best_corr}")
        print(f"Anno {year}: Le 5 statistiche peggio correlate con {stat} sono:\n{worst_corr}")
    else:
        print(f"La statistica {stat} non è presente nel dataset. Controlla l'input")

# Creo una funzione che restituisce l'elenco degli indci di correlazione di una statistica con le altre anno per anno

def correlazioni_annuali(stat):
    for file in files:
        year = year_mapping[file]
        data_frame = pd.read_excel(file)  
        data_frame['Duels'] = data_frame['Won'] + data_frame['Lost']

        data_frame.drop(columns=colonne_da_eliminare, inplace=True)

        corr_matrix = data_frame.corr()

        if stat in corr_matrix.columns:
            sorted_corr = corr_matrix[stat].sort_values(ascending=False)

            print(f"Anno {year}: Le correlazioni per {stat} sono:\n{sorted_corr}\n")
        else:
            print(f"La statistica {stat} non è presente nel dataset dell'anno {year}.")
        
# Creo una seconda funzione che mi dice l'indice di correlazione di due statistiche specifiche
# ed il cambiamento di questo nel corso delle stagioni

def correlazione(stat1,stat2):
 correlations = []

 for file in files:
    
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost']

    data_frame.drop(columns=colonne_da_eliminare, inplace=True)

    corr_matrix = data_frame.corr()

    year = year_mapping[file]

    # Verifica se entrambe le statistiche esistono nella matrice di correlazione
    if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
        correlation_value = corr_matrix.loc[stat1, stat2]
        correlations.append((year, correlation_value))
        print(f"Anno {year}: Indice {correlation_value}")
    else:
        correlations.append((year, None))

# Creazione del DataFrame per il grafico
 if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
    correlation_df = pd.DataFrame(correlations, columns=['Anno', 'Correlazione'])
    
    # Crea il grafico dell'andamento della correlazione nel tempo
    fig = px.line(correlation_df, x='Anno', y='Correlazione', title=f"Correlazione tra {stat1} e {stat2}", markers=True)
    fig.update_xaxes(dtick=1)
    fig.update_yaxes(range = [-1,1], dtick = 0.10)
    fig.show()
    print(f"Il massimo valore dell'indice fra {stat1} e {stat2} è {correlation_df['Correlazione'].max()} il valore minimo è {correlation_df['Correlazione'].min()}")
 else:
    print("Nessun grafico disponibile: non sono state calcolate correlazioni valide. Controlla l'input")

Ci aiutiamo con delle funzioni per plottare l'andamento di questi indici negli anni considerati.

In [ ]:
for column in data_frame.columns:
    if column not in ['Gls','Ast']:
     correlazione('Gls',column)

Notiamo che l'unica coppia che presenta massimo e minimo negativi è Gls e Min si presenta quindi una anticorrelazione. Per le altre 3 coppie si ha correlazione diretta in quanto massimo e minimo sono entrambi positivi.

Ragionando in valore assoluto, la coppia che risulta avere la correlazione più debole è Gls e Age nel 2020. Mentre quella più forte è Gls e Starts nel 2021.

In [ ]:
for column in data_frame.columns:
    if column not in ['Gls','Ast']:
     correlazione('Ast',column)

Anche in questo caso si presenta anticorrelazione con i minuti giocati, infatti anche qui il range è interamente a valori negativi. Tutte le altre hanno intervallo con massimo e minimo positivi e quindi correlazione diretta.

Passando al valore assoluto la coppia con la correlazione più debole è Ast e Min nel 2021. Quella più forte ad 1 è Ast e Starts nel 2020.

2) STATISTICHE DI OPPOSIZIONE

In questa parte ci occupiamo di analizzare le statistiche di opposizione, ovvero quelle che il riguardano il comportamento del giocatore quando è messo a confronto con un avversario. Analizzeremo dati come dribbling, contrasti e conduzioni palla. 

In questa parte son presenti statistiche sulla parte di campo in cui vengono registrati determinati dati. Le analizziamo separatamente in quanto vogliamo capire se c'è un impatto diverso legato all'area di campo in cui vengono registrati.

In [ ]:
correlations = []
years = []

colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad','Age', 'MP', 'Starts', 'Min', 'G+A', 'G-PK', 'PK',
       'PKatt', 'CrdY', 'CrdR', 'xG', 'PrgP', 'PrgR', 'Sh', 'SoT', 'SoT%',
       'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG', 'Cmp', 'Att', 'Cmp%', 'xA', 'KP',
       'PPA', 'CrsPA', 'SCA', 'GCA', 'Succ%','2CrdY', 'Off', 'Crs','Won%']

for file in files:
    
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost'] 
    data_frame.drop(columns=colonne_da_eliminare, inplace=True) 

    corr_matrix = data_frame.corr()

    year = year_mapping[file] 
    years.append(year)

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}", zmin = -1, zmax = 1)
    fig.show()
print(f'Le colonne usate per questa analisi sono {list(data_frame.columns)}')

Visivamente possiamo notare come in questo blocco di statistiche gli indici siano più alti rispetto a quello precedente, essendo la griglia composta da colori più caldi. Quindi ci aspettiamo che i range di correlazione siano tutti positivi.

In [ ]:
for column in ['Mid 3rd', 'Att 3rd', 'Mid 3rd.1', 'Att 3rd.1']:
    correlazione('Gls',column)

Le prime due coppie sono relative ai contrasti, nella parte centrale di campo e quella offensiva. La seconda invece tratta i dribbling, anche qui divisi in parte centrale e offensiva. Per entrambe le categorie notiamo che gli intervalli degli indici sono più elevati per la parte offensiva. Quindi possiamo concludere che la parte di campo in cui vengono fatti dribbling e contrasti ha effettivamente un impatto sui Gol.

In [ ]:
for column in ['Mid 3rd', 'Att 3rd', 'Mid 3rd.1', 'Att 3rd.1']:
    correlazione('Ast',column)

Vediamo che i risultati sono simili a prima quindi il discorso è il medesimo che per i gol. Realizzare dribbling e contrasti nella parte offensiva di campo ha un impatto maggiore sugli assist piuttosto che farli a centrocampo.

In [ ]:
for column in data_frame.columns:
    if column not in ['Gls','Ast','Mid 3rd', 'Att 3rd', 'Mid 3rd.1', 'Att 3rd.1']:
     correlazione('Gls',column)

Vediamo che tutti i minimi sono positivi quindi siamo in presenza solamente di correlazioni dirette.
La coppia con corelazione più debole è Gls e TklW nel 2020 . Quella più forte è Gls e Touches nel 2023.

In [ ]:
for column in data_frame.columns:
    if column not in ['Gls','Ast','Mid 3rd', 'Att 3rd', 'Mid 3rd.1', 'Att 3rd.1']:
     correlazione('Ast',column)

Anche qui siamo solamente in presenza di correlazioni dirette. Rispetto ai gol si hanno correlazioni più forti, come in Carries che ha registrato la correlazione più forte con Ast nel 2021, ma anche alcune più deboli come quella di Won che ha registrato l'indice di correlazione con Ast più basso nel 2022.

3) STATISTICHE DI GIOCO

In questa parte andiamo ad analizzare le statistiche di creazione offensiva, quelle relative ai passaggi ai tiri e alla creazione di chance da gol o di tiro

In [ ]:
correlations = []
years = []

colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad','Age', 'MP', 'Starts', 'Min', 'G+A', 'G-PK', 'PK',
       'PKatt', 'CrdY', 'CrdR', 'SoT%', 'xG', 'xA',
       'G/Sh', 'G/SoT', 'FK', 'G-xG','Cmp%','Tkl', 'TklW', 'Mid 3rd', 'Att 3rd',
       'Touches', 'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ', 'Succ%',
       'Carries', 'TotDist', '2CrdY', 'Fld', 'Duels', 'Won', 'Lost',
       'Won%']

for file in files:
    
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost'] 
    data_frame.drop(columns=colonne_da_eliminare, inplace=True) 

    corr_matrix = data_frame.corr()

    year = year_mapping[file] 
    years.append(year)

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}", zmin = -1, zmax = 1)
    fig.show()
    
print(f'Le colonne usate per questa analisi sono {list(data_frame.columns)}')

In queste matrici vediamo colori molto caldi, quindi ci aspettiamo solo correlazioni dirette. Ad eccezione per una colonna con correlazione molto debole vicinissima allo zero sia per Gol che per Assist relativa a Dist, ovvero la distanza media di tiro. Risultato prevedibile. Analizziamola a parte per dimostrare ciò che abbiamo appena detto.

In [ ]:
for column in ['Gls','Ast']:
    correlazione('Dist',column)

Fra Dist e Gol l'intervallo degli indici di correlazione è negativo siamo in presenza di anticorrelazione molto debole.
Con gli Ast l'intervallo comprende valori negativi e positivi tutti vicini allo zero correlazione quindi, anche in questo caso, molto debole.

In [ ]:
for column in data_frame.columns:
    if column not in ['Gls','Ast','Dist']:
     correlazione('Gls',column)

Come anticipato, siccome tutti i minimi sono positivi, siamo in presenza di correlazioni dirette più o meno forti. La più forte è Gls e SoT nel 2021, il range è interamente molto vicino all'uno quindi siamo in presenza di una correlazione molto forte. Al contrario la correlazione è con CrsPA registrato nel 2019.

In [ ]:
for column in data_frame.columns:
    if column not in ['Gls','Ast','Dist']:
     correlazione('Ast',column)

Siamo in presenza di sole correlazioni dirette. Qua per gli assist la correlazione più forte è con GCA registrata nel 2021. Mentre la più debole è con Off registrato nel 2019.

4) GOL E ASSIST ATTESI

In questa parte finale di analisi ci concentriamo sul concetto specifico di Gol ed Assist attesi

I concetti di gol attesi (xG) e assist attesi (xA) sono metriche avanzate utilizzate nel calcio per misurare la qualità delle azioni da gol e degli assist, tenendo conto di fattori che influenzano la probabilità che si verifichi un gol o un assist. Il valore è compreso tra 0 e 1, dove un valore di 1 indica un evento che ha quasi certezza di verificarsi. mentre 0.1 rappresenta un tiro (ad esempio) molto difficile con poca probabilità di realizzazione.
Nello specifico, xG prende in considerazione aspetti come:
- Posizione del tiro: più vicino alla porta, più alta è la probabilità di segnare.
- Angolo di tiro: tiri da angoli difficili (ad esempio, tiri da fuori area o angoli stretti) hanno una probabilità inferiore di andare in gol.
- Posizione del portiere: se il portiere è fuori posizione o se c'è un errore difensivo, la probabilità di segnare aumenta.

Come per l'xG, anche l'xA si basa su vari fattori che influenzano la probabilità di trasformare un passaggio in un gol, come:
- Tipo di passaggio: passaggi facili da ricevere e controllare (come palloni rasoterra diretti) hanno una probabilità maggiore di diventare assist.
- Distanza dal gol: passaggi effettuati vicino alla porta o da posizioni centrali hanno un xA più alto.
- Posizione del ricevente: se il destinatario del passaggio è in una buona posizione per segnare, l'assist atteso sarà più alto.

In [72]:
colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad','Age', 'MP', 'Starts', 'Min', 'G+A', 'G-PK', 'PK',
       'PKatt', 'CrdY', 'CrdR', 'SoT%',
       'G/Sh', 'G/SoT', 'FK', 'G-xG','Cmp%','Tkl', 'TklW', 'Mid 3rd', 'Att 3rd',
       'Touches', 'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ', 'Succ%',
       'Carries', 'TotDist', '2CrdY', 'Fld', 'Duels', 'Won', 'Lost',
       'Won%']

In [73]:
stat1 = 'Gls'
stat2 = 'xG'
correlazione(stat1,stat2)

Anno 2023: Indice 0.9354220145380396
Anno 2022: Indice 0.9307850335682799
Anno 2021: Indice 0.9406995365569356
Anno 2020: Indice 0.9565836298172762
Anno 2019: Indice 0.9438927059160955


Il massimo valore dell'indice fra Gls e xG è 0.9565836298172762 il valore minimo è 0.9307850335682799


Come potevamo immaginarci dalla spiegazione precedente il range è estremamente vicino ad 1, quindi la correlazione è molto forte fra queste due statistiche. Il risultato è molto comprensibile in quanto dietro a questi dati ci sono studi molto profondi e complessi realizzati anche con l'aiuto di intelligenze artificiali e database pienissimi di dati di miliardi di giocatori presi in diverse annate.

In [74]:
stat1 = 'Ast'
stat2 = 'xA'
correlazione(stat1,stat2)

Anno 2023: Indice 0.7740110619603474
Anno 2022: Indice 0.8057140831372767
Anno 2021: Indice 0.8713995391935595
Anno 2020: Indice 0.8847975606015525
Anno 2019: Indice 0.8607413104725158


Il massimo valore dell'indice fra Ast e xA è 0.8847975606015525 il valore minimo è 0.7740110619603474


La correlazione anche in questo caso è forte. Lo è meno di quanto lo sono gli xG con i Gol in quanto per gli assist c'è la variabile relativa al compagno che può non riuscire a concretizzare un ottimo passaggio.

CONCLUSIONE

La ricerca conferma la nostra tesi. Esistono statistiche che presentano correlazioni forti con Gol ed Assist. Inoltre per ogni coppia di statistiche la differenza fra il minimo ed il massimo valore che assume l'indice mostra una costanza nella correlazione delle statistiche nel corso delle stagioni.

RETI NEURALI

Grazie all'analisi di correlazione abbiamo capito quali statistiche sono correlate con i dati di Gol ed Assist. Daremo quindi in input alla rete neurale quelle statistiche e chiederemo in output, appunto, Gol ed Assist.


Nelle prossime celle andremo ad impostare la rete neurale, importando i file ed addestrandola con i file di input. 
Andiamo a fare prima i gol e nella cella seguente la rete per gli assist

In [ ]:
f1 = "attaccanti_19.xlsx"
f2 = "attaccanti_20.xlsx"
f3 = "attaccanti_21.xlsx"
f4 = "attaccanti_22.xlsx"
f5 = "attaccanti_23.xlsx"

# Elimino le colonne che abbiamo scoperto essere non utili per la nostra analisi

colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad', 'Age', 'MP', 'Starts', 'Min', 'G+A', 'G-PK', 'PK', 'PKatt', 
                        'CrdY', 'CrdR', 'SoT%', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG','Cmp', 'Att','CrsPA', 'Cmp%', 'Tkl',
                        'TklW', 'Mid 3rd', 'Att 3rd', 'Mid 3rd.1','Succ%', 'TotDist', '2CrdY', 'Fld', 'Off','Crs', 'Won', 'Lost', 'Won%']

# Unisco i file Excel
df1 = pd.read_excel(f1)
df2 = pd.read_excel(f2).iloc[1:].reset_index(drop=True)
df3 = pd.read_excel(f3).iloc[1:].reset_index(drop=True)
df4 = pd.read_excel(f4).iloc[1:].reset_index(drop=True)
df_unito = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Rimuovo le colonne
df_unito = df_unito.drop(columns=colonne_da_eliminare, errors='ignore')

df5 = pd.read_excel(f5).iloc[1:].reset_index(drop=True)
df5 = df5.drop(columns=colonne_da_eliminare, errors='ignore')

# Separare le colonne in input e output
X = df_unito.iloc[:, 2:]  # Tutte le colonne eccetto le prime due
y = df_unito.iloc[:, :2]  # Le prime due colonne

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divisione in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Costruzione della rete neurale
model = tf.keras.Sequential([tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
                             tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dense(y_train.shape[1])])

# Compilazione
model.compile(optimizer='adam', loss='mean_squared_error')

# Addestramento
model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_split=0.2)

# Carica il nuovo file e pre-processa i dati
df_new = df5  

# Verifica che le colonne siano allineate con il DataFrame di addestramento
X_new = df_new[X.columns]

# Pre-elabora i dati nuovi
X_new_scaled = scaler.transform(X_new)

# Usa il modello per fare previsioni
y_pred = model.predict(X_new_scaled)

# Imposto pari a zero i valori negativi in quanto è impossibile realizzare un numero negativo di gol
 
y_pred = np.clip(y_pred, 0, None)  # Sostituisci i valori negativi con 0

# Visualizza le previsioni
df_new['Pred_Gls'] = y_pred[:, 0]  # Previsioni per la prima colonna (Gls)
df_new['Pred_Ast'] = y_pred[:, 1]  # Previsioni per la seconda colonna (Ast)

print(df_new[['Gls', 'Ast', 'Pred_Gls', 'Pred_Ast']])

In [ ]:
# Calcola separatamente la somma delle previsioni per Gls e Ast
somma_goal_attesi = df_new['Pred_Gls'].sum()
somma_assist_attesi = df_new['Pred_Ast'].sum()

# Calcola separatamente la somma effettiva dei goal e assist nei dati originali
somma_goal = df_new['Gls'].sum()
somma_assist = df_new['Ast'].sum()
# Risultati dei gol
print(f"La somma dei goal effettivi è {somma_goal}")
print(f"La somma dei goal ottenuti dalla rete neurale è {somma_goal_attesi}")

# Risultati degli assist

print(f"La somma degli assist effettivi è {somma_assist}")
print(f"La somma degli assist ottenuti dalla rete neurale è {somma_assist_attesi}")


In [ ]:
# Errore

diff_gol = abs(somma_goal - somma_goal_attesi)
diff_ast = abs(somma_assist - somma_assist_attesi)

# Trovo l'errore medio dividendo per il numero di giocatori 

avg_err_gol = diff_gol/194  
avg_err_ast = diff_ast/194

print(f"La differenza fra la somma effettiva e quella della rete neurale è di {diff_gol} Gol")
print(f"La differenza fra la somma effettiva e quella della rete neurale è di {diff_ast} Assist")
print(f"L'errore medio per i gol è {avg_err_gol} per giocatore")
print(f"L'errore medio per gli assist è {avg_err_ast} per giocatore")


Aumentando il numero delle epoche ovviamente la differenza fra la previsione della rete neurale e la somma effettiva delle statistiche richieste si assottiglia notevolmente. Il processo richiede più tempo ma restituisce risultati più precisi.